In [1]:
import gzip
import pickle
import os
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from pylab import rcParams
import pandas as pd
import pystan
from matplotlib.ticker import MultipleLocator
from scipy.stats import gaussian_kde
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
rcParams['figure.figsize'] = 16, 10
COMMON_SEED = 1234

def check_convergence(fit, also_print=False):
    report = print
    
    def all_rhat_small_enough(fit):
        return all(dict(fit.summary())['summary'][:, -1] < 1.1)
    
    def max_treedepth_exceeded(fit, max_depth = 10):
        """Check transitions that ended prematurely due to maximum tree depth limit"""
        sampler_params = fit.get_sampler_params(inc_warmup=False)
        depths = [x for y in sampler_params for x in y['treedepth__']]
        n = sum(1 for x in depths if x == max_depth)
        if n > 0:
            report('Run again with max_depth set to a larger value to avoid saturation')        
        N = len(depths)
        report(('{} of {} iterations saturated the maximum tree depth of {}'
               + ' ({}%)').format(n, N, max_depth, 100 * n / N))
        return float(n) / N
    
    def e_bfmi_all_low_enough(fit):
        """
        Checks the energy Bayesian fraction of missing information (E-BFMI).
        E-BFMI below 0.2 indicates you may need to reparameterize your model
        """
        sampler_params = fit.get_sampler_params(inc_warmup=False)
        for chain_num, s in enumerate(sampler_params):
            energies = s['energy__']
            numer = sum((energies[i] - energies[i - 1])**2 for i in range(1, len(energies))) / len(energies)
            denom = np.var(energies)
            if numer / denom < 0.2:
                report('Chain {}: E-BFMI = {}'.format(chain_num, numer / denom))
                report('E-BFMI below 0.2 indicates you may need to reparameterize your model')
                return False
        return True

    def fraction_of_transitions_which_ended_with_divergence(fit):
        """Check transitions that ended with a divergence"""
        sampler_params = fit.get_sampler_params(inc_warmup=False)
        divergent = [x for y in sampler_params for x in y['divergent__']]
        n = sum(divergent)
        N = len(divergent)
        report('{} of {} iterations ended with a divergence ({}%)'
              .format(n, N, 100 * n / N))
        if n > 0:
            report('Try running with larger adapt_delta to remove the divergences')
        return n / N
    
    convergence_results = {
        "all_rhat_small_enough(fit)": all_rhat_small_enough(fit),
        "max_treedepth_exceeded(fit) < 0.02": max_treedepth_exceeded(fit) < 0.02,
        "e_bfmi_all_low_enough(fit)": e_bfmi_all_low_enough(fit),
        "fraction_of_transitions_which_ended_with_divergence(fit) <= 5E-3": 
            fraction_of_transitions_which_ended_with_divergence(fit) <= 5E-3
    }
    report(convergence_results)
    report("##### All convergence checks passed successfully. #####" if all(convergence_results.values()) else
          "##### Some convergence checks failed, see above. #####")

In [3]:
births_2000s_df = pd.read_csv('US_births_2000-2014_SSA.csv')
births_2000s_df_with_index = births_2000s_df\
    .rename(columns={'date_of_month': 'day'})\
    .set_index(pd.to_datetime(
        births_2000s_df.rename(columns={'date_of_month': 'day'})
        [['year', 'month', 'day']]))\
    .assign(weekday_name=lambda df: df.index.weekday_name)\
    .assign(day_of_year=lambda df: df.index.dayofyear)\
    .assign(week_of_year=lambda df: df.index.weekofyear)
births_2000s_df_with_index.tail()

,year,month,day,day_of_week,births,weekday_name,day_of_year,week_of_year
2014-12-27,2014,12,27,6,8656,Saturday,361,52
2014-12-28,2014,12,28,7,7724,Sunday,362,52
2014-12-29,2014,12,29,1,12811,Monday,363,1
2014-12-30,2014,12,30,2,13634,Tuesday,364,1
2014-12-31,2014,12,31,3,11990,Wednesday,365,1


In [4]:
def weekday_priors_by_year_2000():
    df_2000 = births_2000s_df_with_index[lambda df: df.year == 2000]
    return df_2000.groupby('day_of_week').agg(['mean', 'std']).births

weekday_priors_by_year_2000()

,mean,std
day_of_week,,
1,11514.461538,1036.715902
2,12870.846154,790.289954
3,12762.269231,422.905919
4,12735.461538,813.567006
5,12524.500000,634.105530
6,9049.830189,344.643056
7,8014.433962,310.302366


In [5]:
def priors_for_weekend_vs_workday_by_year_2000():
    df_2000 = births_2000s_df_with_index[lambda df: df.year == 2000]
    return df_2000\
        .assign(is_weekend=lambda df: df.day_of_week.isin([6, 7]))\
        .groupby('is_weekend')\
        .agg(['mean', 'std'])\
        .births

priors_for_weekend_vs_workday_by_year_2000()

,mean,std
is_weekend,,
False,12481.507692,909.221295
True,8532.132075,614.062616


In [6]:
def get_train_and_test_for_sliding_window(test_date):
    num_data_points_before_first_test_date = datetime(2014, 1, 1) - datetime(2001, 1, 1)
    df = births_2000s_df_with_index[['day_of_week', 'births']]\
        [lambda df: df.index <= test_date + timedelta(days=6)]\
        [lambda df: df.index >= test_date - num_data_points_before_first_test_date]
    return {
        'x_train': df.day_of_week[:-7].values,
        'y_train': df.births[:-7].values,
        'x_test': df.day_of_week[-7:].values,
        'y_test': df.births[-7:].values
    }
        
get_train_and_test_for_sliding_window(datetime(2014, 1, 1))

{'x_test': array([3, 4, 5, 6, 7, 1, 2]),
 'x_train': array([1, 2, 3, ..., 7, 1, 2]),
 'y_test': array([ 8018, 11171, 12317,  8199,  7174, 11400, 12310]),
 'y_train': array([ 7663, 10635, 12449, ...,  7896, 13096, 12525])}

In [7]:
def rmse(prediction_errors):
    return (prediction_errors ** 2).mean() ** 0.5

In [8]:
model_hier_negbin = pystan.StanModel(model_code='''
data {
    int<lower=0> T; // Number of predictions.
    int<lower=0> N; // Number of data points.
    int y_train[N]; // Train data points.
    int y_test[T]; // Test data points, for log-likelihood.
    int<lower=1,upper=7> x_train[N]; // Weekday indicator for each observed data point.
    int<lower=1,upper=7> x_test[T]; // Weekday indicator for each prediction.
    real<lower=0> prior_mean_weekend;
    real<lower=0> prior_mean_workday;
    real<lower=0> prior_std_weekend;
    real<lower=0> prior_std_workday;
    real<lower=0> V_lambda; // Std. dev. for priors on lambda.
    real<lower=0> V_sigma; // Std. dev. for priors on sigma.
    real<lower=0> V_phi; // Std. dev. for priors on phi.
    real<lower=0> prior_phi;
    real<lower=0> V_daily_phi;
}
parameters {
    real<lower=0> common_phi;
    real<lower=0> phi[7];
    real<lower=0> mu[7];
    real<lower=0> mu_weekend;
    real<lower=0> mu_workday;
    real<lower=0> sigma_weekend;
    real<lower=0> sigma_workday;
}
model {
    common_phi ~ normal(prior_phi, V_phi * prior_phi);
    mu_weekend ~ normal(prior_mean_weekend, V_lambda * prior_std_weekend);
    mu_workday ~ normal(prior_mean_workday, V_lambda * prior_std_workday);
    sigma_weekend ~ normal(prior_std_weekend, V_sigma * prior_std_weekend);
    sigma_workday ~ normal(prior_std_workday, V_sigma * prior_std_workday);
    mu[6] ~ normal(mu_weekend, sigma_weekend);
    mu[7] ~ normal(mu_weekend, sigma_weekend);
    for (i in 1:5) {
        mu[i] ~ normal(mu_workday, sigma_workday);
    }
    phi ~ cauchy(common_phi, V_daily_phi * common_phi);
    y_train ~ neg_binomial_2(mu[x_train], phi[x_train]);
}
generated quantities {
    real loglik;
    vector[T] y_pred;
    loglik = 0;
    for (t in 1:T) {
        y_pred[t] = neg_binomial_2_rng(mu[x_test[t]], phi[x_test[t]]);
        loglik += neg_binomial_2_lpmf(y_test[t] | mu[x_test[t]], phi[x_test[t]]);
    }
}
''')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1f5534f9f16a420dad5dd0b0607ea807 NOW.


In [9]:
prior_stds = weekday_priors_by_year_2000()['std'].values
prior_mu = weekday_priors_by_year_2000()['mean'].values
prior_phi = (prior_mu ** 2) / (prior_stds ** 2 - prior_mu)
print("prior_stds:", prior_stds, "prior_mu:", prior_mu, "prior_phi:", prior_phi, sep="\n")

prior_stds:
[1036.71590189  790.28995385  422.90591896  813.56700585  634.10553028
  344.64305592  310.30236553]
prior_mu:
[11514.46153846 12870.84615385 12762.26923077 12735.46153846
 12524.5         9049.83018868  8014.43396226]
prior_phi:
[124.69400825 270.82246617 980.66297605 249.85061788 402.66186638
 746.37900747 727.64108437]


In [ ]:
# Second week of 2014.
data = get_train_and_test_for_sliding_window(datetime(2014, 1, 1) + timedelta(days=7))
fit = model_hier_negbin.sampling(seed=COMMON_SEED, iter=500, data=dict(
        V_daily_phi=0.02,
        prior_phi=prior_phi.mean(),
        V_phi=1,
        V_lambda=4,
        V_sigma=0.2,
        prior_mean_weekend=priors_for_weekend_vs_workday_by_year_2000()[lambda df: df.index == True]['mean'].values[0],
        prior_mean_workday=priors_for_weekend_vs_workday_by_year_2000()[lambda df: df.index == False]['mean'].values[0],
        prior_std_workday=priors_for_weekend_vs_workday_by_year_2000()[lambda df: df.index == False]['std'].values[0],
        prior_std_weekend=priors_for_weekend_vs_workday_by_year_2000()[lambda df: df.index == True]['std'].values[0],
        x_train=data['x_train'],
        x_test=data['x_test'],
        y_train=data['y_train'],
        y_test=data['y_test'],
        T=7,
        N=len(data['x_train'])))

In [ ]:
check_convergence(fit)

In [ ]:
fit

In [ ]:
fit.plot();

In [ ]:
rmse(fit.extract()["y_pred"].mean(axis=0) - data["y_test"])

In [10]:
def hier_negbin_data_extractor(data):
    return dict(
        V_daily_phi=0.02,
        prior_phi=prior_phi.mean(),
        V_phi=1,
        V_lambda=4,
        V_sigma=0.2,
        prior_mean_weekend=priors_for_weekend_vs_workday_by_year_2000()[lambda df: df.index == True]['mean'].values[0],
        prior_mean_workday=priors_for_weekend_vs_workday_by_year_2000()[lambda df: df.index == False]['mean'].values[0],
        prior_std_workday=priors_for_weekend_vs_workday_by_year_2000()[lambda df: df.index == False]['std'].values[0],
        prior_std_weekend=priors_for_weekend_vs_workday_by_year_2000()[lambda df: df.index == True]['std'].values[0],
        x_train=data['x_train'],
        x_test=data['x_test'],
        y_train=data['y_train'],
        y_test=data['y_test'],
        T=7,
        N=len(data['x_train']))

In [11]:
def fit_stan_model_on_sliding_windows(iterations, stan_data_extractor, persist_path, stan_model, num_windows):
    if not os.path.exists(persist_path):
        os.mkdir(persist_path)
    all_prediction_errors = []
    for i in range(num_windows):
        data = get_train_and_test_for_sliding_window(datetime(2014, 1, 1) + timedelta(days=i * 7))
        fit = stan_model.sampling(seed=COMMON_SEED, data=stan_data_extractor(data), iter=iterations, chains=1,
                                 control=dict(adapt_delta=0.99, max_treedepth=20))
        print(fit)
        check_convergence(fit, also_print=True)
        pred_err = fit.extract()['y_pred'].mean(axis=0) - data['y_test']
        all_prediction_errors.append(pred_err)
        with gzip.open(os.path.join(persist_path, 'fit%d_summary.pkl.gz' % i), 'wb') as f:
            pickle.dump(fit.summary(), f)
        with gzip.open(os.path.join(persist_path, 'fit%d_extract.pkl.gz' % i), 'wb') as f:
            pickle.dump(fit.extract(), f)         
        with gzip.open(os.path.join(persist_path, 'fit%d_pred_err.pkl.gz' % i), 'wb') as f:
            pickle.dump(pred_err, f)
        with gzip.open(os.path.join(persist_path, 'fit%d_y_test.pkl.gz' % i), 'wb') as f:
            pickle.dump(data['y_test'], f)
        with gzip.open(os.path.join(persist_path, 'fit%d.pkl.gz' % i), 'wb') as f:
            pickle.dump(fit, f)                        
    return rmse(np.concatenate(all_prediction_errors))

In [12]:
%%time

# Quick check: one time window.
fit_stan_model_on_sliding_windows(
    500,
    hier_negbin_data_extractor,
    'fit_hier_negbin_one_phi', 
    model_hier_negbin,
    1)

Inference for Stan model: anon_model_1f5534f9f16a420dad5dd0b0607ea807.
1 chains, each with iter=500; warmup=250; thin=1; 
post-warmup draws per chain=250, total post-warmup draws=250.

                mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
common_phi    172.59    2.09  14.45 146.67 161.96 173.62 182.61 200.77     48   1.02
phi[0]          85.4    0.31   4.45  76.37  82.86  85.14  88.14  95.15    206    1.0
phi[1]        171.93    1.28   10.8 150.16  164.5  172.5 179.38 191.85     71   1.02
phi[2]        184.56     0.9  10.77 162.58  178.5 184.23 191.35 206.66    142    1.0
phi[3]          96.7    0.36   5.08   87.3  93.13   96.1 100.44 107.95    203    1.0
phi[4]        146.36    0.76   9.33 127.32 140.39 145.84 151.99  164.3    152    1.0
phi[5]        294.76    1.38  17.64  260.6  283.8 293.72 305.93  335.6    163    1.0
phi[6]        387.63     2.0   21.1 342.52 375.37  387.6 399.48 429.88    111    1.0
mu[0]          1.2e4    3.24   51.3  1.2e4  1.2e4 

/home/inonpe/jupyter/dtu_bda_project/env/lib/python3.5/site-packages/ipykernel_launcher.py:22: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.


CPU times: user 5min 52s, sys: 470 ms, total: 5min 52s
Wall time: 5min 51s


2048.2154705814382

In [13]:
%%time

fit_stan_model_on_sliding_windows(
    500,
    hier_negbin_data_extractor,
    'fit_hier_negbin_one_phi', 
    model_hier_negbin,
    52)

Inference for Stan model: anon_model_1f5534f9f16a420dad5dd0b0607ea807.
1 chains, each with iter=500; warmup=250; thin=1; 
post-warmup draws per chain=250, total post-warmup draws=250.

                mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
common_phi    172.59    2.09  14.45 146.67 161.96 173.62 182.61 200.77     48   1.02
phi[0]          85.4    0.31   4.45  76.37  82.86  85.14  88.14  95.15    206    1.0
phi[1]        171.93    1.28   10.8 150.16  164.5  172.5 179.38 191.85     71   1.02
phi[2]        184.56     0.9  10.77 162.58  178.5 184.23 191.35 206.66    142    1.0
phi[3]          96.7    0.36   5.08   87.3  93.13   96.1 100.44 107.95    203    1.0
phi[4]        146.36    0.76   9.33 127.32 140.39 145.84 151.99  164.3    152    1.0
phi[5]        294.76    1.38  17.64  260.6  283.8 293.72 305.93  335.6    163    1.0
phi[6]        387.63     2.0   21.1 342.52 375.37  387.6 399.48 429.88    111    1.0
mu[0]          1.2e4    3.24   51.3  1.2e4  1.2e4 

/home/inonpe/jupyter/dtu_bda_project/env/lib/python3.5/site-packages/ipykernel_launcher.py:22: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.


Inference for Stan model: anon_model_1f5534f9f16a420dad5dd0b0607ea807.
1 chains, each with iter=500; warmup=250; thin=1; 
post-warmup draws per chain=250, total post-warmup draws=250.

                mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
common_phi    168.02    1.41  11.71 145.97 160.16  168.2 176.37 189.26     69    1.0
phi[0]         87.06    0.42   4.96  77.75  83.68   86.7  90.14  99.22    140   1.03
phi[1]        170.31    0.95   9.45 152.03 163.23 170.23 177.36 188.32    100    1.0
phi[2]        176.36    0.88   9.54 158.87 170.07 175.85 182.95 195.67    118    1.0
phi[3]         95.86    0.39   5.32  85.55   92.1  95.95   99.8 106.21    182    1.0
phi[4]        147.77    0.87   8.73 131.06 141.53 147.78 154.28 164.05    100    1.0
phi[5]         296.4    1.05  16.53  266.6 284.34 296.46 308.11 329.64    250    1.0
phi[6]        387.22    1.39  21.91 350.72 369.69 387.75 402.16 432.97    250   1.01
mu[0]          1.2e4     3.7  58.47  1.2e4  1.2e4 

Inference for Stan model: anon_model_1f5534f9f16a420dad5dd0b0607ea807.
1 chains, each with iter=500; warmup=250; thin=1; 
post-warmup draws per chain=250, total post-warmup draws=250.

                mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
common_phi    171.94    1.03  10.13  150.3 165.77 172.88 177.77 192.96     96    1.0
phi[0]         87.28    0.33   4.53  78.96  83.73  87.34  90.28  96.02    184    1.0
phi[1]        172.18    0.78   8.51 156.13 165.89 173.16 177.61 186.93    118    1.0
phi[2]        175.15    0.86   8.59 157.02 169.98 174.91 179.86 193.89     99    1.0
phi[3]         96.33    0.35   5.51  86.99   92.0  96.13 100.31 107.64    250    1.0
phi[4]         147.0    0.81   9.25 129.72 140.33 147.27 153.12 164.96    130   1.01
phi[5]        296.99    1.11  17.53 263.94 284.95 295.86 309.32 332.52    250    1.0
phi[6]        388.56    1.36  21.56 350.68  372.3  387.3 404.04 432.03    250    1.0
mu[0]          1.2e4     2.9  45.84  1.2e4  1.2e4 

Inference for Stan model: anon_model_1f5534f9f16a420dad5dd0b0607ea807.
1 chains, each with iter=500; warmup=250; thin=1; 
post-warmup draws per chain=250, total post-warmup draws=250.

                mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
common_phi    167.65    0.97  10.65 145.57  159.8 167.55 175.71 186.76    121   1.01
phi[0]         86.48     0.3   4.68  78.02  82.99  86.49  89.94  95.59    250    1.0
phi[1]        169.69    0.64   8.87 154.94 163.15 169.22  175.8 188.54    191    1.0
phi[2]        173.75    0.64    8.1 157.21 168.94  173.2 178.87  191.6    158    1.0
phi[3]         95.82    0.35   5.25  86.12  92.66  95.41  99.37 107.04    222    1.0
phi[4]        148.75    0.75   8.82 131.07 142.85 149.38 155.49 164.28    139    1.0
phi[5]        297.98    1.33  17.18 265.09 286.89 298.32  308.8 331.96    168    1.0
phi[6]        388.52    1.52  19.87 350.61 374.69 388.43 402.45  426.3    171    1.0
mu[0]          1.2e4    3.22  50.92  1.2e4  1.2e4 

...

Inference for Stan model: anon_model_1f5534f9f16a420dad5dd0b0607ea807.
1 chains, each with iter=500; warmup=250; thin=1; 
post-warmup draws per chain=250, total post-warmup draws=250.

                mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
common_phi    168.84    1.24  10.43 147.66 162.09 169.48  175.8 188.59     71   1.02
phi[0]         87.26    0.37   5.35  77.33  83.72  86.85  91.37  98.29    212    1.0
phi[1]        170.04    0.86   8.84 152.83 164.34 170.45 176.09 186.96    106   1.01
phi[2]        174.06    0.79   8.95 155.22 167.93 174.31 179.76 190.09    127    1.0
phi[3]          95.8    0.43   5.12  84.68  92.29  96.02  98.86 105.13    142    1.0
phi[4]        142.95    0.71   8.68 125.47 137.84 142.77 148.06 163.83    150    1.0
phi[5]         306.3    1.68  16.01 274.55 296.44 307.17 315.21 340.88     91   1.05
phi[6]        406.65    1.78  23.22 365.57  390.1 405.27 422.17 464.57    171    1.0
mu[0]          1.2e4     3.3  52.13  1.2e4  1.2e4 

Inference for Stan model: anon_model_1f5534f9f16a420dad5dd0b0607ea807.
1 chains, each with iter=500; warmup=250; thin=1; 
post-warmup draws per chain=250, total post-warmup draws=250.

                mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
common_phi    169.98    1.87  12.69  142.0 163.16 171.15 178.21 191.72     46   1.01
phi[0]         86.35    0.47   4.96  75.39  83.37  86.57  89.75  94.33    110   1.01
phi[1]        170.87    0.88   8.79 153.51 165.51 171.19  177.5 186.51    100   1.02
phi[2]         176.4    0.94    9.8 155.57 170.03 177.17 183.55  194.7    108   1.01
phi[3]         95.98    0.34   4.78  86.35  92.47  96.44   99.4 104.69    202    1.0
phi[4]        142.36    0.72   9.29 125.95 135.33 142.11 148.82 160.53    165   1.02
phi[5]        309.23    1.42   18.3 274.29 297.63 309.88 317.91 350.44    167    1.0
phi[6]        404.46    1.69  23.51  361.6 388.39  402.5 422.26 454.83    193    1.0
mu[0]          1.2e4    2.73  43.23  1.2e4  1.2e4 

Inference for Stan model: anon_model_1f5534f9f16a420dad5dd0b0607ea807.
1 chains, each with iter=500; warmup=250; thin=1; 
post-warmup draws per chain=250, total post-warmup draws=250.

                mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
common_phi    168.68    1.51  11.37 146.35 161.27 169.17 176.18 189.11     57   1.01
phi[0]         86.46    0.46   5.46   77.0  82.45  86.19  90.62  96.95    140    1.0
phi[1]        169.87     0.9   9.03 153.02 163.68 169.71 175.85 189.07    101   1.02
phi[2]        175.25    0.69   9.23 158.38 168.53 175.21 181.41 192.92    179   1.02
phi[3]         94.48    0.38   4.74  85.26   90.7  94.46  97.93 103.19    153    1.0
phi[4]        141.75    0.69   8.49 126.05 135.72 141.95 146.53 159.52    152    1.0
phi[5]        305.13     1.1  17.39 272.44 293.13 304.51 318.06 342.51    250    1.0
phi[6]        408.29    1.68  25.06 361.29 390.31 408.54 423.31 455.55    222    1.0
mu[0]          1.2e4    2.92  46.14  1.2e4  1.2e4 

CPU times: user 8h 46min 59s, sys: 46.2 s, total: 8h 47min 46s
Wall time: 8h 46min 16s


898.590679443399